In [1]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode

from sys import exit

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')

In [2]:
casos_raw = pd.read_excel('novos_casos_raw.xlsx', dtype={'rs':str, 'ibge_residencia':str})

In [3]:
casos_raw['hash'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)
casos_raw['hash_less'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']-1) + row['mun_resid']).replace(" ",""), axis=1)
casos_raw['hash_more'] = casos_raw.apply(lambda row: (row['paciente'] + str(row['idade']+1) + row['mun_resid']).replace(" ",""), axis=1)

In [4]:
casos_raw = casos_raw.sort_values(by='paciente')

In [5]:
# casos_raw.loc[casos_raw.duplicated(subset='hash', keep=False)]

In [6]:
print(f"casos novos: {casos_raw.shape[0]}")
print(f"casos novos duplicados: {casos_raw.loc[casos_raw.duplicated(subset='hash')].shape[0]}")
casos_raw = casos_raw.drop_duplicates(subset='hash')
dropar = casos_raw.loc[casos_raw['data_liberacao'].isnull()]
print(f"casos novos sem diagnóstico: {dropar.shape[0]}")
# casos_raw = casos_raw.drop(index=dropar.index)
dropar = casos_raw.loc[casos_raw['nome_exame'].isnull()]
print(f"casos novos sem laboratório: {dropar.shape[0]}")
# casos_raw = casos_raw.drop(index=dropar.index)
dropar = casos_raw.loc[casos_raw['sexo'].isnull()]
print(f"casos novos sem sexo: {dropar.shape[0]}")
# casos_raw = casos_raw.drop(index=dropar.index)
print(f"casos novos validos: {casos_raw.shape[0]}")

casos novos: 2561
casos novos duplicados: 0
casos novos sem diagnóstico: 599
casos novos sem laboratório: 37
casos novos sem sexo: 0
casos novos validos: 2561


In [7]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]

In [8]:
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)
casos_confirmados['hash'] = casos_confirmados.apply(lambda row: (row['nome'] + str(row['idade']) + row['mun_resid']).replace(" ",""), axis=1)

In [9]:
index_casos_duplicados = casos_raw.loc[casos_raw['hash'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a mesma idade: {len(index_casos_duplicados)}")
index_casos_duplicados_idade_less = casos_raw.loc[casos_raw['hash_less'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade - 1: {len(index_casos_duplicados_idade_less)}")
index_casos_duplicados_idade_more = casos_raw.loc[casos_raw['hash_more'].isin(casos_confirmados['hash'])].index.to_list()
print(f"casos já em casos com a idade + 1: {len(index_casos_duplicados_idade_more)}")
print(f"dentre os quais {len(set(index_casos_duplicados_idade_more).intersection(index_casos_duplicados)) + len(set(index_casos_duplicados_idade_less).intersection(index_casos_duplicados))} são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos")
index_duplicados = list(set(index_casos_duplicados + index_casos_duplicados_idade_less + index_casos_duplicados_idade_more))
print(f"sendo assim, {len(index_duplicados)} casos que já se encontram na planilha serão removidos")

casos já em casos com a mesma idade: 1334
casos já em casos com a idade - 1: 17
casos já em casos com a idade + 1: 0
dentre os quais 1 são casos em comum, o que leva a crer que estão duplicados na planilha já com idade a mais ou idade a menos
sendo assim, 1350 casos que já se encontram na planilha serão removidos


In [10]:
print(f"então, de {len(casos_raw)} casos baixados hoje  {len(casos_raw)-len(index_duplicados)} serão adicionados")
casos_raw = casos_raw.drop(index=index_duplicados)

então, de 2561 casos baixados hoje  1211 serão adicionados


In [11]:
casos_raw['data_com'] = [ "" ] * len(casos_raw)

In [12]:
novos_casos = casos_raw[['paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'nome_exame','data_notificacao','data_liberacao','data_com','data_1o_sintomas']]
novos_casos.to_excel('novos_casos.xlsx', index=False)